# Run a simulation

In [7]:
import numpy as np
from numba import njit, prange

### Calculate particle forces using bondlist

In [8]:
@njit(parallel=True)
def calculate_nodal_forces(bondlist, x, u, d, c, cell_volume,
                           s0, s1, sc, beta, f_x, f_y, f_z,
                           node_force):
    """
    Calculate particle forces - employs bondlist

    Parameters
    ----------
    bondlist : ndarray (int)
        Array of pairwise interactions (bond list)
    x : ndarray (float)
        Material point coordinates in the reference configuration
    u : ndarray (float)
        Nodal displacement
    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed
    c : float
        Bond stiffness

    Returns
    -------
    node_force : ndarray (float)
        Nodal force array
    d : ndarray (float)
        Bond damage (softening parameter). The value of d will range from 0
        to 1, where 0 indicates that the bond is still in the elastic range,
        and 1 represents a bond that has failed

    Notes
    -----
    """
    n_bonds = np.shape(bondlist)[0]

    for k_bond in prange(n_bonds):

        node_i = bondlist[k_bond, 0] - 1
        node_j = bondlist[k_bond, 1] - 1

        xi_x = x[node_j, 0] - x[node_i, 0]
        xi_y = x[node_j, 1] - x[node_i, 1]
        xi_z = x[node_j, 2] - x[node_i, 2]

        xi_eta_x = xi_x + (u[node_j, 0] - u[node_i, 0])
        xi_eta_y = xi_y + (u[node_j, 1] - u[node_i, 1])
        xi_eta_z = xi_z + (u[node_j, 2] - u[node_i, 2])

        xi = np.sqrt(xi_x**2 + xi_y**2 + xi_z**2)
        y = np.sqrt(xi_eta_x**2 + xi_eta_y**2 + xi_eta_z**2)
        stretch = (y - xi) / xi

        # TODO: allow the user to load different constitutive models or define
        # a new law that describes the interaction between two particles
        d[k_bond] = trilinear_constitutive_model(stretch, s0, s1, sc,
                                                 d[k_bond], beta)

        f = stretch * c * (1 - d[k_bond]) * cell_volume
        f_x[k_bond] = f * xi_eta_x / y
        f_y[k_bond] = f * xi_eta_y / y
        f_z[k_bond] = f * xi_eta_z / y

    # Reduce bond forces to particle forces
    for k_bond in range(n_bonds):

        node_i = bondlist[k_bond, 0] - 1
        node_j = bondlist[k_bond, 1] - 1

        node_force[node_i, 0] += f_x[k_bond]
        node_force[node_j, 0] -= f_x[k_bond]
        node_force[node_i, 1] += f_y[k_bond]
        node_force[node_j, 1] -= f_y[k_bond]
        node_force[node_i, 2] += f_z[k_bond]
        node_force[node_j, 2] -= f_z[k_bond]

    return node_force, d

### Update particle positions

In [9]:
@njit(parallel=True)
def update_nodal_positions(node_force, u, v, a, damping,
                           node_density, dt, 
                           bc_flag, bc_magnitude, bc_unit_vector):
    """
    Update particle positions using an Euler-Cromer time integration scheme
    
    Parameters
    ----------
    u : ndarray (float)
        Particle displacement
    v : ndarray (float)
        Particle velocity
    a : ndarray (float)
        Particle acceleration

    Returns
    -------

    Notes
    -----

    """

    n_nodes = np.shape(node_force)[0]
    n_dimensions = np.shape(node_force)[1]

    for node_i in prange(n_nodes):
        for dof in range(n_dimensions):
            a[node_i, dof] = (node_force[node_i, dof] - damping * v[node_i, dof]) / node_density
            v[node_i, dof] = v[node_i, dof] + (a[node_i, dof] * dt)
            u[node_i, dof] = u[node_i, dof] + (v[node_i, dof] * dt)
            
            # TODO: how should boundary conditions be applied?
            if bc_flag[node_i, dof] != 0:
                u[node_i, dof] = bc_magnitude * bc_unit_vector[node_i, dof]

    return u, v

### Run simulation

In [10]:
def run_simulation():
    
    calculate_nodal_forces()
    update_nodal_positions()